In [ ]:
import os
os.environ["HF_HOME"] = "/root/autodl-tmp/"
# export HF_HOME="/root/autodl-tmp/"

In [ ]:
# https://github.com/OpenBMB/MiniCPM-V/blob/main/README_zh.md

In [ ]:
!pip install flash_attn

In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer


In [ ]:
! export ALL_PROXY=socks5://127.0.0.1:7891
! export http_proxy=http://127.0.0.1:7890
! export https_proxy=http://127.0.0.1:7890
! export HTTP_PROXY=http://127.0.0.1:7890
! export HTTPS_PROXY=http://127.0.0.1:7890
os.environ['http_proxy']="http://127.0.0.1:7890"
os.environ['https_proxy']="http://127.0.0.1:7890"
os.environ['HTTP_PROXY']="http://127.0.0.1:7890"
os.environ['HTTPS_PROXY']="http://127.0.0.1:7890"

In [ ]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True,
                                    attn_implementation='sdpa', torch_dtype=torch.bfloat16) # sdpa or flash_attention_2, no eager
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)


In [ ]:
!pip install -U bitsandbytes

In [ ]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6-int4', trust_remote_code=True,
                                    attn_implementation='sdpa', torch_dtype=torch.bfloat16) # sdpa or flash_attention_2, no eager
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6-int4', trust_remote_code=True)

In [40]:
image_path = "./HK.XG.04.JPG"

In [41]:
question = 'What is the item brought? and total paid amount, and the item is food?  please reply in Chinese and also output in json format'

In [48]:
image = Image.open(image_path).convert('RGB')

msgs = [{'role': 'user', 'content': [image, question]}]

"""
res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)
print(res)
"""

## if you want to use streaming, please make sure sampling=True and stream=True
## the model.chat will return a generator
res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.9,
    stream=True
)

generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')

{
  "item_brought": "HONDA海膽餅",
  "total_paid_amount": "$34.4 HKD",
  "is_food": true
}

这段文字说明了购买的物品是食品，并且总共支付了$34.4港币。

In [47]:
#print(generated_text)

In [ ]:
"""
  请您描述这个图片, 店铺名称是什么？买了什么东西， 数量是多少， 商品总金额是多少？一共支付了多少？
  What is the item brought? and total paid amount, and the item is food?  please reply in Chinese and also output in json format
"""